# Capture the Flag (RL - Policy Gradient)

- Seung Hyun Kim
- skim449@illinois.edu

## Implementation Details

- Actor-critic
- On-Policy
- 

### Sampling
- [ ] Mini-batch to update 'average' gradient
- [ ] Experience Replay for Random Sampling
- [x] Importance Sampling
    
### Policy Gradient
- [x] MA-A3C

### Stability and Reducing Variance
- [x] Gradient clipping
- [ ] Normalized Reward/Advantage
- [x] Target Network
- [ ] TRPO
- [ ] PPO

### Multiprocessing
- [ ] Synchronous Training (A2C)
- [x] Asynchronous Training (A3C)

### Applied Training Methods:
- [ ] Self-play
- [ ] Batch Policy

## Notes

- This notebook includes:
    - Building the structure of policy driven network.
    - Training with/without render
    - Saver that save model and weights to ./model directory
    - Writer that will record some necessary datas to ./logs

- This notebook does not include:
    - Simulation with RL policy
        - The simulation can be done using policy_RL.py
    - cap_test.py is changed appropriately.
    
## References :
- https://github.com/awjuliani/DeepRL-Agents/blob/master/Vanilla-Policy.ipynb (source)
- https://www.youtube.com/watch?v=PDbXPBwOavc
- https://github.com/lilianweng/deep-reinforcement-learning-gym/blob/master/playground/policies/actor_critic.py (source)
- https://github.com/spro/practical-pytorch/blob/master/reinforce-gridworld/reinforce-gridworld.ipynb
- https://arxiv.org/pdf/1706.02275.pdf

## TODO:

- Try to add experience buffer

!rm -rf logs/Critic_sharing/ model/Critic_sharing

In [1]:
TRAIN_NAME='Critic_sharing'
LOG_PATH='./logs/'+TRAIN_NAME
MODEL_PATH='./model/' + TRAIN_NAME
GPU_CAPACITY=0.95 # gpu capacity in percentage

In [2]:
import os

import multiprocessing
import threading

import tensorflow as tf
import matplotlib.pyplot as plt
%matplotlib inline

import time
import gym
import gym_cap
import gym_cap.envs.const as CONST
import numpy as np
import random
import math

# the modules that you can use to generate the policy. 
import policy.random
import policy.roomba
import policy.policy_RL
import policy.zeros

# Data Processing Module
from utility.dataModule import one_hot_encoder
from utility.utils import MovingAverage as MA
from utility.utils import discount_rewards
from utility.buffer import Trajectory

from network.critic_sharing import Critic_sharing as Network

%load_ext autoreload
%autoreload 2

## Hyperparameters

In [3]:
# Replay Variables
total_episodes= 200000
max_ep = 150
update_frequency = 32
policy_rotation = 128

# Saving Related
save_network_frequency = 1200
save_stat_frequency = 128
moving_average_step = 128

# Training Variables
lr_a = 5e-5
lr_c = 4e-4


gamma = 0.98 # discount_factor

# Env Settings
MAP_SIZE = 20
VISION_RANGE = 19 # What decide the network size !!!
VISION_dX, VISION_dY = 2*VISION_RANGE+1, 2*VISION_RANGE+1
in_size = [None,VISION_dX,VISION_dY,6]
num_policy_pool = 8
nenv = 8

# Asynch Settings
global_scope = 'global'

RED_POLICY = policy.zeros

## Environment Setting

In [4]:
if not os.path.exists(MODEL_PATH):
    os.makedirs(MODEL_PATH)
    
#Create a directory to save episode playback gifs to
if not os.path.exists(LOG_PATH):
    os.makedirs(LOG_PATH)

In [5]:
action_space = 5
n_agent = 4

## A3C Network Structure

<img src="https://cdn-images-1.medium.com/max/1600/1*YtnGhtSAMnnHSL8PvS7t_w.png" width="450">

- Network is given in network.ActorCritic

## Environments

<img src="https://cdn-images-1.medium.com/max/1600/1*Hzql_1t0-wwDxiz0C97AcQ.png" width="450">
<img src="https://lilianweng.github.io/lil-log/assets/images/MADDPG.png" width="450">

In [6]:
global_rewards = MA(moving_average_step)
global_ep_rewards = MA(moving_average_step)
global_length = MA(moving_average_step)
global_succeed = MA(moving_average_step)
global_episodes = 0

# Launch the session
gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=GPU_CAPACITY, allow_growth=True)

sess = tf.Session(config=tf.ConfigProto(gpu_options=gpu_options))
#sess = tf.Session()
progbar = tf.keras.utils.Progbar(total_episodes,interval=1)

## Worker

In [7]:
class Worker(object):
    def __init__(self, name, global_network, sess, global_step=0):
        # Initialize Environment worker
        self.env = gym.make("cap-v0").unwrapped
        self.env.reset(map_size=MAP_SIZE,
                       policy_red=RED_POLICY.PolicyGen(self.env.get_map, self.env.get_team_red))
        self.name = name
        
        # Create AC Network for Worker
        self.network = Network(in_size=in_size,
                     action_size=action_space,
                     num_agent=n_agent,
                     lr_actor=lr_a,
                     lr_critic=lr_c,
                     scope=self.name,
                     entropy_beta = 0.01,
                     sess=sess,
                     global_network=global_network,
                     num_policy_pool=num_policy_pool,
                     allow_same_policy=True)
        
        self.sess=sess
        
    def work(self, saver, writer):
        global global_rewards, global_ep_rewards, global_episodes, global_length, global_succeed
        traj_depth = 5
        local_ep = 0
                
        with self.sess.as_default(), self.sess.graph.as_default():
            while global_episodes < total_episodes:
                local_ep += 1
                self.env.reset()
                s1 = one_hot_encoder(self.env._env, self.env.get_team_blue, VISION_RANGE)
                a1, a_probs1, v1 = self.network.get_action_critic(s1)
                
                # parameters 
                ep_r = 0 # Episodic Reward
                prev_r = 0
                rc = 0
                is_alive = [ag.isAlive for ag in self.env.get_team_blue]

                trajs = [Trajectory(depth=traj_depth) for _ in range(n_agent)]
                
                for step in range(max_ep+1):
                    # Iteration
                    a, a_probs, v0 = a1, a_probs1, v1
                    was_alive = is_alive
                    s0 = s1
                    prev_r = rc
                    
                    s1, rc, d, _ = self.env.step(a)
                    s1 = one_hot_encoder(self.env._env, self.env.get_team_blue, VISION_RANGE)
                    is_alive = [ag.isAlive for ag in self.env.get_team_blue]

                    r = (rc - prev_r)
                    if step == max_ep and d == False:
                        r = -100
                        rc = -100
                        d = True

                    r /= 100.0
                    ep_r += r

                    if d:
                        v1 = [0.0 for _ in range(len(self.env.get_team_blue))]
                    else:
                        a1, a_probs1, v1 = self.network.get_action_critic(s1)

                    # push to buffer
                    for idx, agent in enumerate(self.env.get_team_blue):
                        if was_alive[idx]:
                            trajs[idx].append([s0[idx],
                                              a[idx],
                                              r,
                                              v0[idx],
                                              a_probs[idx][a[idx]]
                                            ])


                    
                    if local_ep % update_frequency == 0 or d:
                        alosses, closs = self.train(trajs, sess, v1)
                        trajs = [Trajectory(depth=traj_depth) for _ in range(n_agent)]

                    if d:
                        break
                        
                global_ep_rewards.append(ep_r)
                global_rewards.append(rc)
                global_length.append(step)
                global_succeed.append(self.env.blue_win)
                global_episodes += 1
                self.sess.run(global_step_next)
                progbar.update(global_episodes)
                if global_episodes % save_stat_frequency == 0 and global_episodes != 0:
                    summary = tf.Summary()
                    summary.value.add(tag='Records/mean_reward', simple_value=global_rewards())
                    summary.value.add(tag='Records/mean_length', simple_value=global_length())
                    summary.value.add(tag='Records/mean_succeed', simple_value=global_succeed())
                    summary.value.add(tag='Records/mean_episode_reward', simple_value=global_ep_rewards())
                    summary.value.add(tag='summary/actor_loss_mean', simple_value=np.mean(alosses))
                    summary.value.add(tag='summary/critic_loss', simple_value=closs)
                    writer.add_summary(summary,global_episodes)
                    writer.flush()
                if global_episodes % save_network_frequency == 0:
                    saver.save(self.sess, MODEL_PATH+'/ctf_policy.ckpt', global_step=global_episodes)
                    
                if local_ep % policy_rotation == 0:
                    self.network.select_policy()
                    self.network.pull_global()

                        
    def train(self, trajs, sess, bootstrap=0.0):
        obs = []
        act = []
        adv = []
        td  = []
        beta= []
        alosses = []
        closses = []
        for idx, traj in enumerate(trajs):
            if len(traj) == 0:
                alosses.append(None)
                closses.append(None)
                obs.append(None)
                act.append(None)
                adv.append(None)
                td.append(None)
                beta.append(None)
                continue
            observations = traj[0]
            actions = traj[1]
            rewards = np.array(traj[2])
            values = np.array(traj[3])
            behavior_policy = np.array(traj[4])
            
            rewards_ext = np.append(rewards, [bootstrap[idx]])
            #discounted_rewards = discount_rewards(rewards_ext,gamma)[:-1]
            value_ext = np.append(values, [bootstrap[idx]])
            td_targets = rewards + gamma * value_ext[1:]
            advantages = rewards + gamma * value_ext[1:] - value_ext[:-1]
            advantages = discount_rewards(advantages,gamma)
            
            obs.append(observations)
            act.append(actions)
            adv.append(advantages)
            td.append(td_targets)
            beta.append(behavior_policy)
            
        alosses, closs = self.network.update_full(obs,act,adv,td,beta)
        
        self.network.pull_global()
        
        return alosses, closs
    

## Run

In [8]:
# Global Network
global_step = tf.Variable(0, trainable=False, name='global_step')
global_step_next = tf.assign_add(global_step, 1)
global_ac = Network(in_size=in_size,
                    action_size=action_space,
                    scope=global_scope,
                    num_agent=n_agent,
                    num_policy_pool=num_policy_pool,
                    sess=sess)

# Local workers
workers = []
# loop for each workers
for idx in range(nenv):
    name = 'W_%i' % idx
    print(f'worker: {name} initializing')
    workers.append(Worker(name, global_ac, sess, global_step=global_step))
saver = tf.train.Saver(max_to_keep=3)
writer = tf.summary.FileWriter(LOG_PATH, sess.graph)
    
ckpt = tf.train.get_checkpoint_state(MODEL_PATH)
if ckpt and tf.train.checkpoint_exists(ckpt.model_checkpoint_path):
    saver.restore(sess, ckpt.model_checkpoint_path)
    print("Load Model : ", ckpt.model_checkpoint_path)
else:
    sess.run(tf.global_variables_initializer())
    print("Initialized Variables")
    
coord = tf.train.Coordinator()
worker_threads = []
global_episodes = sess.run(global_step)

saver.save(sess, MODEL_PATH+'/ctf_policy.ckpt', global_step=global_episodes)

# Summarize
for var in tf.trainable_variables(scope=global_scope):
    tf.summary.histogram(var.name, var)
merged_summary_op = tf.summary.merge_all()

worker: W_0 initializing


/home/namsong/anaconda3/envs/py36/lib/python3.6/site-packages/gym/envs/registration.py:14: PkgResourcesDeprecationWarning: Parameters to load are deprecated.  Call .resolve and .require separately.
  result = entry_point.load(False)


worker: W_1 initializing
worker: W_2 initializing
worker: W_3 initializing
worker: W_4 initializing
worker: W_5 initializing
worker: W_6 initializing
worker: W_7 initializing
INFO:tensorflow:Restoring parameters from ./model/Critic_sharing/ctf_policy.ckpt-99600
Load Model :  ./model/Critic_sharing/ctf_policy.ckpt-99600
INFO:tensorflow:Summary name global/actor0/Conv/weights:0 is illegal; using global/actor0/Conv/weights_0 instead.
INFO:tensorflow:Summary name global/actor0/Conv/biases:0 is illegal; using global/actor0/Conv/biases_0 instead.
INFO:tensorflow:Summary name global/actor0/Conv_1/weights:0 is illegal; using global/actor0/Conv_1/weights_0 instead.
INFO:tensorflow:Summary name global/actor0/Conv_1/biases:0 is illegal; using global/actor0/Conv_1/biases_0 instead.
INFO:tensorflow:Summary name global/actor0/fully_connected/weights:0 is illegal; using global/actor0/fully_connected/weights_0 instead.
INFO:tensorflow:Summary name global/actor0/fully_connected/biases:0 is illegal; usi

In [ ]:
for worker in workers:
    job = lambda: worker.work(saver, writer)
    t = threading.Thread(target=job)
    t.start()
    worker_threads.append(t)
coord.join(worker_threads)

 99601/200000 [=============>................] - ETA: 54s